In [1]:
import cv2
import numpy as np
from openvino.runtime import Core

PRECISION = "FP32"
MODEL_NAME = "colorization-v2"
MODEL_PATH = "C:/Users/Pedro/public/colorization-v2/FP32/colorization-v2.xml"
DATA_DIR = "data"
DEVICE = "GPU"

# openvino inference
ie = Core()
model = ie.read_model(model=MODEL_PATH)
compiled_model = ie.compile_model(model=model, device_name=DEVICE)
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
N, C, H, W = list(input_layer.shape)

def colorize(gray_img: np.ndarray):
    
    # Preprocess
    h_in, w_in, _ = gray_img.shape
    #print('shape gray', gray_img.shape)
    img_rgb = gray_img.astype(np.float32) / 255
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
    img_l = cv2.resize(img_lab.copy(), (W, H))[:, :, 0]
    #print('shape lab, l', img_lab.shape, img_l_rs.shape)

    # Inference
    inputs = np.expand_dims(img_l, axis=[0, 1])
    #print('shape inputs', inputs.shape)
    res = compiled_model([inputs])[output_layer]
    update_res = np.squeeze(res)

    # Post-process
    out = update_res.transpose((1, 2, 0))
    out = cv2.resize(out, (w_in, h_in))
    
    # adding a single channel
    img_lab = img_lab[:, :, 0][:, :, np.newaxis]
    img_lab_out = np.concatenate((img_lab, out), axis=2)

    # clip values beyond 0-1
    img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)
    colorized_image = (cv2.resize(img_bgr_out, (w_in, h_in))
                       * 255).astype(np.uint8)
    
    return colorized_image

cap = cv2.VideoCapture('video_original.mp4')
writer=None

fps = cap.get(cv2.CAP_PROP_FPS)
print('Frame rate of the video', fps)

print('Reading frames')
while cap.isOpened():
    
    ret, raw_image = cap.read()

    if not ret:
        break

    # if image has more than one channel
    if raw_image.shape[2] > 1:
        gray_img = cv2.cvtColor(
                     cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY),
                     cv2.COLOR_GRAY2RGB)
    else:
        gray_img = cv2.cvtColor(raw_image, cv2.COLOR_GRAY2RGB)

    # color the frame
    print('Coloring the frame....')
    color_image = colorize(gray_img)
    
    if writer is None:

        print('Writing to a video file..')
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        (H0,W0) = color_image.shape[:2]
        writer = cv2.VideoWriter("video_colorizado_colorization-v2.mp4", fourcc, int(fps), (W0,H0), True)

    #save video
    writer.write(color_image)

    key = ord('q')
    if cv2.waitKey(1) == key:
        break
        
cap.release()
cv2.destroyAllWindows()

Frame rate of the video 25.0
Reading frames
Coloring the frame....
Writing to a video file..
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring t

Se realiza misma operación pero con modelo aplicable a imágenes y videos "colorization-siggraph" para evaluar diferencias con modelo "colorization-v2".

In [2]:
PRECISION = "FP32"
MODEL_NAME = "colorization-siggraph"
MODEL_PATH = "C:/Users/Pedro/public/colorization-siggraph/FP32/colorization-siggraph.xml"
DATA_DIR = "data"
DEVICE = "GPU"

# openvino inference
ie = Core()
model = ie.read_model(model=MODEL_PATH)
compiled_model = ie.compile_model(model=model, device_name=DEVICE)
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
N, C, H, W = list(input_layer.shape)

def colorize(gray_img: np.ndarray):
    
    # Preprocess
    h_in, w_in, _ = gray_img.shape
    #print('shape gray', gray_img.shape)
    img_rgb = gray_img.astype(np.float32) / 255
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
    img_l = cv2.resize(img_lab.copy(), (W, H))[:, :, 0]
    #print('shape lab, l', img_lab.shape, img_l_rs.shape)

    # Inference
    inputs = np.expand_dims(img_l, axis=[0, 1])
    #print('shape inputs', inputs.shape)
    res = compiled_model([inputs])[output_layer]
    update_res = np.squeeze(res)

    # Post-process
    out = update_res.transpose((1, 2, 0))
    out = cv2.resize(out, (w_in, h_in))
    
    # adding a single channel
    img_lab = img_lab[:, :, 0][:, :, np.newaxis]
    img_lab_out = np.concatenate((img_lab, out), axis=2)

    # clip values beyond 0-1
    img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)
    colorized_image = (cv2.resize(img_bgr_out, (w_in, h_in))
                       * 255).astype(np.uint8)
    
    return colorized_image

cap = cv2.VideoCapture('video_original.mp4')
writer=None

fps = cap.get(cv2.CAP_PROP_FPS)
print('Frame rate of the video', fps)

print('Reading frames')
while cap.isOpened():
    
    ret, raw_image = cap.read()

    if not ret:
        break

    # if image has more than one channel
    if raw_image.shape[2] > 1:
        gray_img = cv2.cvtColor(
                     cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY),
                     cv2.COLOR_GRAY2RGB)
    else:
        gray_img = cv2.cvtColor(raw_image, cv2.COLOR_GRAY2RGB)

    # color the frame
    print('Coloring the frame....')
    color_image = colorize(gray_img)
    
    if writer is None:

        print('Writing to a video file..')
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        (H0,W0) = color_image.shape[:2]
        writer = cv2.VideoWriter("video_colorizado_colorization-siggraph.mp4", fourcc, int(fps), (W0,H0), True)

    #save video
    writer.write(color_image)

    key = ord('q')
    if cv2.waitKey(1) == key:
        break
        
cap.release()
cv2.destroyAllWindows()

Frame rate of the video 25.0
Reading frames
Coloring the frame....
Writing to a video file..
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring t

In [3]:
# Implementacion de mejora de video_original.
import cv2
import numpy as np

# Función para aplicar superresolución
def aplicar_superresolucion(frame):
    # Función cv2.resize para aumentar la resolución
    superres_frame = cv2.resize(frame, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    return superres_frame

# Función para aplicar corrección de contraste
def aplicar_correccion_contraste(frame):
    # Función cv2.equalizeHist para ecualizar el histograma
    frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    contraste_frame = cv2.equalizeHist(frame_gris)
    return cv2.cvtColor(contraste_frame, cv2.COLOR_GRAY2BGR)

# Función para aplicar reducción de ruido
def aplicar_reduccion_ruido(frame):
    # Función cv2.fastNlMeansDenoising para reducir el ruido
    reduccion_ruido_frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    return reduccion_ruido_frame

# Cargar el video original
video_original = cv2.VideoCapture('video_original.mp4')

# Obtener las propiedades del video original
fps = int(video_original.get(cv2.CAP_PROP_FPS))
frame_width = int(video_original.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(video_original.get(cv2.CAP_PROP_FRAME_COUNT))

# Crear un objeto VideoWriter para guardar el video mejorado
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_mejorado = cv2.VideoWriter('video_mejorado.mp4', fourcc, fps, (frame_width, frame_height))

# Procesar cada fotograma del video original
while True:
    ret, frame = video_original.read()
    if not ret:
        break

    # Aplicar superresolución al frame
    frame_superresolucion = aplicar_superresolucion(frame)

    # Aplicar corrección de contraste al frame
    frame_correccion_contraste = aplicar_correccion_contraste(frame)

    # Aplicar reducción de ruido al frame
    frame_reduccion_ruido = aplicar_reduccion_ruido(frame)

    # Redimensionar frame_superresolucion para que tenga las mismas dimensiones que los otros marcos
    frame_superresolucion = cv2.resize(frame_superresolucion, (frame_correccion_contraste.shape[1], frame_correccion_contraste.shape[0]))

    # Combinar los frames procesados para obtener el resultado final
    frame_mejorado = cv2.addWeighted(frame_superresolucion, 1/3, frame_correccion_contraste, 1/3, 0)
    frame_mejorado = cv2.addWeighted(frame_mejorado, 1, frame_reduccion_ruido, 1/3, 0)

    # Escribir el frame procesado en el video mejorado
    video_mejorado.write(frame_mejorado)

# Liberar los recursos
video_original.release()
video_mejorado.release()
cv2.destroyAllWindows()

In [4]:
PRECISION = "FP32"
MODEL_NAME = "colorization-v2"
MODEL_PATH = "C:/Users/Pedro/public/colorization-v2/FP32/colorization-v2.xml"
DATA_DIR = "data"
DEVICE = "GPU"

# openvino inference
ie = Core()
model = ie.read_model(model=MODEL_PATH)
compiled_model = ie.compile_model(model=model, device_name=DEVICE)
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
N, C, H, W = list(input_layer.shape)

def colorize(gray_img: np.ndarray):

    # Preprocess
    h_in, w_in, _ = gray_img.shape
    #print('shape gray', gray_img.shape)
    img_rgb = gray_img.astype(np.float32) / 255
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
    img_l = cv2.resize(img_lab.copy(), (W, H))[:, :, 0]
    #print('shape lab, l', img_lab.shape, img_l_rs.shape)

    # Inference
    inputs = np.expand_dims(img_l, axis=[0, 1])
    #print('shape inputs', inputs.shape)
    res = compiled_model([inputs])[output_layer]
    update_res = np.squeeze(res)

    # Post-process
    out = update_res.transpose((1, 2, 0))
    out = cv2.resize(out, (w_in, h_in))
    
    # adding a single channel
    img_lab = img_lab[:, :, 0][:, :, np.newaxis]
    img_lab_out = np.concatenate((img_lab, out), axis=2)

    # clip values beyond 0-1
    img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)
    colorized_image = (cv2.resize(img_bgr_out, (w_in, h_in))
                       * 255).astype(np.uint8)

    return colorized_image

cap = cv2.VideoCapture('video_mejorado.mp4')
writer=None

fps = cap.get(cv2.CAP_PROP_FPS)
print('Frame rate of the video', fps)

print('Reading frames')
while cap.isOpened():

    ret, raw_image = cap.read()

    if not ret:
        break

    # if image has more than one channel
    if raw_image.shape[2] > 1:
        gray_img = cv2.cvtColor(
                     cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY),
                     cv2.COLOR_GRAY2RGB)
    else:
        gray_img = cv2.cvtColor(raw_image, cv2.COLOR_GRAY2RGB)

    # color the frame
    print('Coloring the frame....')
    color_image = colorize(gray_img)
    
    if writer is None:

        print('Writing to a video file..')
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        (H0,W0) = color_image.shape[:2]
        writer = cv2.VideoWriter("video_mejorado_colorization-v2.mp4", fourcc, int(fps), (W0,H0), True)

    #save video
    writer.write(color_image)

    key = ord('q')
    if cv2.waitKey(1) == key:
        break
        
cap.release()
cv2.destroyAllWindows()

Frame rate of the video 25.0
Reading frames
Coloring the frame....
Writing to a video file..
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring t

Se observó que el modelo "colorization-v2" tuvo mejor calidad, por lo que se continuará solo con este modelo.

Se realizará prueba con video de mejora calidad que el anterior, que corresponde a película muda chilena de 1925 "El húsar de la muerte"

In [5]:
PRECISION = "FP32"
MODEL_NAME = "colorization-v2"
MODEL_PATH = "C:/Users/Pedro/public/colorization-v2/FP32/colorization-v2.xml"
DATA_DIR = "data"
DEVICE = "GPU"

# openvino inference
ie = Core()
model = ie.read_model(model=MODEL_PATH)
compiled_model = ie.compile_model(model=model, device_name=DEVICE)
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
N, C, H, W = list(input_layer.shape)

def colorize(gray_img: np.ndarray):

    # Preprocess
    h_in, w_in, _ = gray_img.shape
    #print('shape gray', gray_img.shape)
    img_rgb = gray_img.astype(np.float32) / 255
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
    img_l = cv2.resize(img_lab.copy(), (W, H))[:, :, 0]
    #print('shape lab, l', img_lab.shape, img_l_rs.shape)

    # Inference
    inputs = np.expand_dims(img_l, axis=[0, 1])
    #print('shape inputs', inputs.shape)
    res = compiled_model([inputs])[output_layer]
    update_res = np.squeeze(res)

    # Post-process
    out = update_res.transpose((1, 2, 0))
    out = cv2.resize(out, (w_in, h_in))
    
    # adding a single channel
    img_lab = img_lab[:, :, 0][:, :, np.newaxis]
    img_lab_out = np.concatenate((img_lab, out), axis=2)

    # clip values beyond 0-1
    img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)
    colorized_image = (cv2.resize(img_bgr_out, (w_in, h_in))
                       * 255).astype(np.uint8)

    return colorized_image

cap = cv2.VideoCapture('husar_de-la_muerte.mp4')
writer=None

fps = cap.get(cv2.CAP_PROP_FPS)
print('Frame rate of the video', fps)

print('Reading frames')
while cap.isOpened():
    
    ret, raw_image = cap.read()

    if not ret:
        break

    # if image has more than one channel
    if raw_image.shape[2] > 1:
        gray_img = cv2.cvtColor(
                     cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY),
                     cv2.COLOR_GRAY2RGB)
    else:
        gray_img = cv2.cvtColor(raw_image, cv2.COLOR_GRAY2RGB)

    # color the frame
    print('Coloring the frame....')
    color_image = colorize(gray_img)

    if writer is None:

        print('Writing to a video file..')
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        (H0,W0) = color_image.shape[:2]
        writer = cv2.VideoWriter("husar_de-la_muerte_colorization-v2.mp4", fourcc, int(fps), (W0,H0), True)

    #save video
    writer.write(color_image)

    key = ord('q')
    if cv2.waitKey(1) == key:
        break
    
cap.release()
cv2.destroyAllWindows()

Frame rate of the video 29.97
Reading frames
Coloring the frame....
Writing to a video file..
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring 

Se implementa mejora a video "El húsar de la muerte" para evaluar cambios en la colorización.

In [6]:
import cv2
import numpy as np

# Función para aplicar superresolución
def aplicar_superresolucion(frame):
    # Función cv2.resize para aumentar la resolución
    superres_frame = cv2.resize(frame, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    return superres_frame

# Función para aplicar corrección de contraste
def aplicar_correccion_contraste(frame):
    # Función cv2.equalizeHist para ecualizar el histograma
    frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    contraste_frame = cv2.equalizeHist(frame_gris)
    return cv2.cvtColor(contraste_frame, cv2.COLOR_GRAY2BGR)

# Función para aplicar reducción de ruido
def aplicar_reduccion_ruido(frame):
    # Función cv2.fastNlMeansDenoising para reducir el ruido
    reduccion_ruido_frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    return reduccion_ruido_frame

# Cargar el video original
video_original = cv2.VideoCapture('husar_de-la_muerte.mp4')

# Obtener las propiedades del video original
fps = int(video_original.get(cv2.CAP_PROP_FPS))
frame_width = int(video_original.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(video_original.get(cv2.CAP_PROP_FRAME_COUNT))

# Crear un objeto VideoWriter para guardar el video mejorado
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_mejorado = cv2.VideoWriter('husar_de-la_muerte_mejorado.mp4', fourcc, fps, (frame_width, frame_height))

# Procesar cada fotograma del video original
while True:
    ret, frame = video_original.read()
    if not ret:
        break

    # Aplicar superresolución al frame
    frame_superresolucion = aplicar_superresolucion(frame)

    # Aplicar corrección de contraste al frame
    frame_correccion_contraste = aplicar_correccion_contraste(frame)

    # Aplicar reducción de ruido al frame
    frame_reduccion_ruido = aplicar_reduccion_ruido(frame)

    # Redimensionar frame_superresolucion para que tenga las mismas dimensiones que los otros marcos
    frame_superresolucion = cv2.resize(frame_superresolucion, (frame_correccion_contraste.shape[1], frame_correccion_contraste.shape[0]))

    # Combinar los frames procesados para obtener el resultado final
    frame_mejorado = cv2.addWeighted(frame_superresolucion, 1/3, frame_correccion_contraste, 1/3, 0)
    frame_mejorado = cv2.addWeighted(frame_mejorado, 1, frame_reduccion_ruido, 1/3, 0)

    # Escribir el frame procesado en el video mejorado
    video_mejorado.write(frame_mejorado)

# Liberar los recursos
video_original.release()
video_mejorado.release()
cv2.destroyAllWindows()

In [7]:
PRECISION = "FP32"
MODEL_NAME = "colorization-v2"
MODEL_PATH = "C:/Users/Pedro/public/colorization-v2/FP32/colorization-v2.xml"
DATA_DIR = "data"
DEVICE = "GPU"

# openvino inference
ie = Core()
model = ie.read_model(model=MODEL_PATH)
compiled_model = ie.compile_model(model=model, device_name=DEVICE)
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
N, C, H, W = list(input_layer.shape)

def colorize(gray_img: np.ndarray):

    # Preprocess
    h_in, w_in, _ = gray_img.shape
    #print('shape gray', gray_img.shape)
    img_rgb = gray_img.astype(np.float32) / 255
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
    img_l = cv2.resize(img_lab.copy(), (W, H))[:, :, 0]
    #print('shape lab, l', img_lab.shape, img_l_rs.shape)

    # Inference
    inputs = np.expand_dims(img_l, axis=[0, 1])
    #print('shape inputs', inputs.shape)
    res = compiled_model([inputs])[output_layer]
    update_res = np.squeeze(res)

    # Post-process
    out = update_res.transpose((1, 2, 0))
    out = cv2.resize(out, (w_in, h_in))
    
    # adding a single channel
    img_lab = img_lab[:, :, 0][:, :, np.newaxis]
    img_lab_out = np.concatenate((img_lab, out), axis=2)

    # clip values beyond 0-1
    img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)
    colorized_image = (cv2.resize(img_bgr_out, (w_in, h_in))
                       * 255).astype(np.uint8)

    return colorized_image

cap = cv2.VideoCapture('husar_de-la_muerte_mejorado.mp4')
writer=None

fps = cap.get(cv2.CAP_PROP_FPS)
print('Frame rate of the video', fps)

print('Reading frames')
while cap.isOpened():

    ret, raw_image = cap.read()

    if not ret:
        break

    # if image has more than one channel
    if raw_image.shape[2] > 1:
        gray_img = cv2.cvtColor(
                     cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY),
                     cv2.COLOR_GRAY2RGB)
    else:
        gray_img = cv2.cvtColor(raw_image, cv2.COLOR_GRAY2RGB)

    # color the frame
    print('Coloring the frame....')
    color_image = colorize(gray_img)
    
    if writer is None:

        print('Writing to a video file..')
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        (H0,W0) = color_image.shape[:2]
        writer = cv2.VideoWriter("husar_de-la_muerte_mejorado_colorization-v2.mp4", fourcc, int(fps), (W0,H0), True)

    #save video
    writer.write(color_image)

    key = ord('q')
    if cv2.waitKey(1) == key:
        break
        
cap.release()
cv2.destroyAllWindows()

Frame rate of the video 29.0
Reading frames
Coloring the frame....
Writing to a video file..
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring the frame....
Coloring t